In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import nltk

nltk.download('gutenberg')
from nltk.corpus import gutenberg



[nltk_data] Downloading package gutenberg to /home/gris/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
bible_words = list(set(gutenberg.words('bible-kjv.txt')))
bible_sents = gutenberg.sents('bible-kjv.txt')

word_to_vect = dict()
dict_size = len(bible_words)

for i in range(dict_size):
    word_to_vect[bible_words[i]] = [0] * dict_size
    word_to_vect[bible_words[i]][i] = 1

input_sents = list()
i = 0
for sent in bible_sents:
    i+=1
    input_sents.append(list())
    for word in sent:
        input_sents[-1].append(word_to_vect[word])
    if i % 100 == 0:
        print(i)
input_sents = torch.tensor(input_sents)
print('complete')

In [25]:
#https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/
#I understand the math, but pytorch provides nn.RNN
#in the future I would like to analyze that library
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim

        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        out = nn.softmax(out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(batch_size, self.hidden_dim)
        return hidden
hidden_dim = dict_size
n_layers = 1

In [26]:
model = Model(dict_size, dict_size, hidden_dim)
n_epochs = 100
lr=0.01

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    output, hidden = model(input_sents)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

AttributeError: 'list' object has no attribute 'size'